In [1]:
%pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
def print_output(docs):
    for i, doc in enumerate(docs):
        print(
            ">>>{}: The output is: {}. \n\nThe metadata is {} \n\n".format(
                i + 1,
                (
                    doc.page_content[:100] + "..."
                    if len(doc.page_content) > 100
                    else doc.page_content
                ),
                doc.metadata,
            )
        )

In [4]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
db = FAISS.load_local(
    "summarizer_index", embeddings_model, allow_dangerous_deserialization=True
)

## Perform semantic search

In [5]:
text = """
worked with partners
"""
docs = db.similarity_search(text)

In [6]:
print_output(docs)

>>>1: The output is: |    |                                                                                              .... 

The metadata is {} 


>>>2: The output is: Forever grateful to the big Nokia family for this opportunity, it was worth meeting you and I hope y.... 

The metadata is {} 


>>>3: The output is: |    | relevant articles that match the partner's                                                   .... 

The metadata is {} 


>>>4: The output is: | 2                                                    | Contribute to the release of the first team.... 

The metadata is {} 




In [7]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [8]:
model_name = "ibm-granite/granite-3.0-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
# CPU device with no GPU
# model = AutoModelForCausalLM.from_pretrained(model_name)
# device_id = "cpu"

In [10]:
# Metal Performance Shaders (MPS) for Apple GPUs
device_id = "mps"
device = torch.device(device_id)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device_id,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,  # important to avoid generation errors
)

In [12]:
prompt = hub.pull("rlm/rag-prompt")
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "human",
#             """You are an assistant for question-answering tasks. 
# Use the following pieces of retrieved context to answer the question. 
# If you don't know the answer, just say that you don't know. 
# Use four sentences maximum and keep the answer concise.
# Question: {question} 
# Context: {context} 
# Answer: <your answer here>
# Word count: <total word count>
# """,
#         )
#     ]
# )

/Users/dmartino/projects/AI/learning/chat-with-your-data-using-chatgpt-3806110/venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [14]:
llm = HuggingFacePipeline(pipeline=pipeline)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# combine multiple steps in a single chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()  # convert the chat message to a string
)

In [15]:
question = """
You are the person for which we provide the development and individual item reports
and the LinkedIn profile description.
Generate a profile description for yourself.
The profile must include:
- one paragraph with the description of professional role, highlighting the competences
- one paragraph with a comma separated list of recent achievements
- one paragraph with a comma separated list of skills

Use first person as "I am" instead of "He is".
Use generic terms instead of specific examples: "DevOps" instead of "Jenkins".
Avoid to mention customer names like Temenos.
Use skills categorizies when possible.
At the end of each sentence add a list of the reference source files.
At the end of the response, include a count of words.
"""

In [16]:
for chunk in rag_chain.stream(
    question
):
    print(chunk, end="", flush=True)



Professional Role:

I am a Principal Software Engineer at Red Hat, with a strong focus on software architecture, design, and development. I have expertise in various technologies such as Java, Spring, Hibernate, REST, and Agile (Scrum). I am also proficient in TDD and MDD. My role involves leading a development team of up to 200+ people worldwide and continuously earning their trust through my technical excellence and proven track record of top-ranking performance evaluations.

Recent Achievements:

1. Learned about the Red Hat product portfolio, gaining a good understanding of its features and capabilities, particularly OpenShift Container Platform.
2. Understood Red Hat divisions, relationships, and organizations, as well as the Open Source development process and productization concerns.
3. Completed C. Wright's learning courses on AI and started Feast-related trainings.
4. Started using the Red Hat learning day to collaborate on selected projects.
5. Created comprehensible and ap